In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## 데이터 크롤링
* 스탯티즈(http://www.statiz.co.kr/main.php)에서 2017~2022년도의 5월부터 9월까지 경기의 각종 데이터를 가져온다.
* 이때 무승부 경기는 데이터 샘플에 포함하지 않는다.


In [6]:
data_set = {'타석':[],'타수':[],'안타':[],'홈런':[],'볼넷':[],'사구':[],'삼진':[],
            '땅볼':[],'플라이':[],'팀타율':[],'출루율':[],'잔루':[],'상대선발투수ERA':[],'상대선발투수WHIP':[]}
label = {'스코어':[]} 

def data_sample_crawling():
    try:
        #박스스코어 엔터
        driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[1]/div/div[3]/div/div[2]/table/tbody/tr/td/a[4]").send_keys(Keys.ENTER)

        #A팀 타자데이터 크롤링
        tmp = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[1]/div/div")
        data = tmp.find_element(By.CLASS_NAME,"colhead_stz").text
        A_team_hit = list(map(float,data.split()[2:]))
        A_GOB =  (A_team_hit[3]+A_team_hit[6]+A_team_hit[7])/(A_team_hit[0]+A_team_hit[6]+A_team_hit[7]+A_team_hit[10])
        A_AVG = A_team_hit[3]/A_team_hit[1]


        #B팀 타자 데이터 크롤링
        tmp = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[2]/div/div")
        data = tmp.find_element(By.CLASS_NAME,"colhead_stz").text
        B_team_hit = list(map(float,data.split()[2:]))     
        B_GOB =  (B_team_hit[3]+B_team_hit[6]+B_team_hit[7])/(B_team_hit[0]+B_team_hit[6]+B_team_hit[7]+B_team_hit[10])
        B_AVG = B_team_hit[3]/B_team_hit[1]

        #A팀 상대선발 투수 데이터 크롤링
        A_team_pitcher = []
        for tmp in range(15,20):
            data = driver.find_element(By.XPATH,f"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[3]/div/div/div[2]/table/tbody/tr[2]/td[{str(tmp)}]").text
            A_team_pitcher.append(data)
        A_team_pitcher = list(map(float,A_team_pitcher))

        #B팀 상대선발 투수 데이터 크롤링
        B_team_pitcher = []
        for tmp in range(15,20):
            data = driver.find_element(By.XPATH,f"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div[4]/div/div/div[2]/table/tbody/tr[2]/td[{str(tmp)}]").text
            B_team_pitcher.append(data)
        B_team_pitcher = list(map(float,B_team_pitcher))

        #레이블
        tmp = driver.find_element(By.CLASS_NAME,"callout")
        win_lose = tmp.find_element(By.XPATH,"/html/body/div[1]/div[1]/div/section[2]/div/div[1]/div/div[1]/div/div[2]/span").text

        a,b =map(int,win_lose.split(':'))
        if a > b:
            tmp_label=0
        elif a < b:
            tmp_label=1
        else:
            return #무승부는 훈련샘플로 포함X
    except:
        return
    
    #######################################################################################################################

    #A팀 타자 데이터 삽입
    data_set['타석'].append(A_team_hit[0])
    data_set['타수'].append(A_team_hit[1])
    data_set['안타'].append(A_team_hit[3])
    data_set['홈런'].append(A_team_hit[4])
    data_set['볼넷'].append(A_team_hit[6])
    data_set['사구'].append(A_team_hit[7])
    data_set['삼진'].append(A_team_hit[8])
    data_set['땅볼'].append(A_team_hit[9])
    data_set['플라이'].append(A_team_hit[10])
    data_set['팀타율'].append(A_AVG)
    data_set['출루율'].append(A_GOB)
    data_set['잔루'].append(A_team_hit[13])

    #B팀 타자 데이터 삽입
    data_set['타석'].append(B_team_hit[0])
    data_set['타수'].append(B_team_hit[1])
    data_set['안타'].append(B_team_hit[3])
    data_set['홈런'].append(B_team_hit[4])
    data_set['볼넷'].append(B_team_hit[6])
    data_set['사구'].append(B_team_hit[7])
    data_set['삼진'].append(B_team_hit[8])
    data_set['땅볼'].append(B_team_hit[9])
    data_set['플라이'].append(B_team_hit[10])
    data_set['팀타율'].append(B_AVG)
    data_set['출루율'].append(B_GOB)
    data_set['잔루'].append(B_team_hit[13])


    #A팀 상대선발 투수 데이터 삽입
    data_set['상대선발투수ERA'].append(B_team_pitcher[0])
    data_set['상대선발투수WHIP'].append(B_team_pitcher[1])

     #B팀 상대선발 투수 데이터 삽입
    data_set['상대선발투수ERA'].append(A_team_pitcher[0])
    data_set['상대선발투수WHIP'].append(A_team_pitcher[1])

    #레이블 삽입
    label['스코어'].append(a)
    label['스코어'].append(b)

    print(A_team_hit)
    print(B_team_hit)
    print(A_team_pitcher)
    print(B_team_pitcher)        


In [7]:

driver = webdriver.Chrome('chromedriver')
driver.get(f"http://www.statiz.co.kr/schedule.php?opt=5&sy=2019")
#training data 모으기
for year in range(2019,2023):
    for month in range(5,10):
        for i in range(2,7):
            for j in range(1,8):
                for k in range(1,6):
                    try:
                        driver.find_element(By.XPATH,f"/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div/div/div/div[2]/table/tbody/tr[{str(i)}]/td[{str(j)}]/div[3]/a[{str(k)}]").send_keys(Keys.ENTER)  
                    except:
                        continue
                    data_sample_crawling()
                    driver.get(f"http://www.statiz.co.kr/schedule.php?opt={str(month)}&sy={str(year)}")


#엑셀 형태로 저장
data_set_df = pd.DataFrame(data_set)
label_df = pd.DataFrame(label)
data_set_df.to_excel(f'baseball_data.xlsx')
label_df.to_excel(f'label.xlsx')

[40.0, 34.0, 5.0, 9.0, 2.0, 5.0, 4.0, 1.0, 9.0, 8.0, 8.0, 141.0, 0.0, 19.0, 0.265, 0.888, 1.19, 0.093, 0.01]
[35.0, 30.0, 7.0, 6.0, 1.0, 5.0, 3.0, 1.0, 3.0, 13.0, 8.0, 147.0, 0.0, 19.0, 0.2, 0.652, 1.5, 0.593, 2.56]
[4.22, 1.36, 0.9, 0.028, 0.77]
[4.78, 1.46, 0.9, -0.079, 0.33]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.124)
Stacktrace:
Backtrace:
	Ordinal0 [0x0052ACD3+2075859]
	Ordinal0 [0x004BEE61+1633889]
	Ordinal0 [0x003BB7BD+571325]
	Ordinal0 [0x003A2E1A+470554]
	Ordinal0 [0x0040AA0B+895499]
	Ordinal0 [0x0041AC96+961686]
	Ordinal0 [0x00407136+880950]
	Ordinal0 [0x003DFEFD+720637]
	Ordinal0 [0x003E0F3F+724799]
	GetHandleVerifier [0x007DEED2+2769538]
	GetHandleVerifier [0x007D0D95+2711877]
	GetHandleVerifier [0x005BA03A+521194]
	GetHandleVerifier [0x005B8DA0+516432]
	Ordinal0 [0x004C682C+1665068]
	Ordinal0 [0x004CB128+1683752]
	Ordinal0 [0x004CB215+1683989]
	Ordinal0 [0x004D6484+1729668]
	BaseThreadInitThunk [0x756600F9+25]
	RtlGetAppContainerNamedObjectPath [0x773B7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x773B7B8E+238]
